In [1]:
from datasets import load_dataset

ds = load_dataset("JacobLinCool/VoiceBank-DEMAND-16k", use_auth_token=True, cache_dir="/storage/hdd0/data/mayank_dataset")

/home/mayank.mds2023/miniconda3/envs/aml/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1
Reusing dataset parquet (/storage/hdd0/data/mayank_dataset/JacobLinCool___parquet/JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 618.77it/s]


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64'

import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

1.4.0
10.1
True


In [3]:
ds.keys()

dict_keys(['test', 'train'])

In [4]:
print(ds)

DatasetDict({
    test: Dataset({
        features: ['id', 'clean', 'noisy'],
        num_rows: 824
    })
    train: Dataset({
        features: ['id', 'clean', 'noisy'],
        num_rows: 11572
    })
})


In [5]:
ds['test'][0]

{'id': 'p232_001',
 'clean': {'path': 'p232_001.wav',
  'array': array([ 0.00213623,  0.00323486,  0.00244141, ..., -0.00576782,
         -0.00543213, -0.00354004]),
  'sampling_rate': 16000},
 'noisy': {'path': 'p232_001.wav',
  'array': array([ 0.01589966,  0.02572632,  0.02059937, ..., -0.00421143,
         -0.00469971, -0.00354004]),
  'sampling_rate': 16000}}

In [8]:
for example in ds['train']:
    print(example)
    break

{'id': 'p226_001', 'clean': {'path': 'p226_001.wav', 'array': array([-0.00286865, -0.00497437, -0.00469971, ..., -0.00201416,
       -0.00152588, -0.00021362]), 'sampling_rate': 16000}, 'noisy': {'path': 'p226_001.wav', 'array': array([-0.00283813, -0.00488281, -0.00473022, ...,  0.006073  ,
        0.00564575,  0.00540161]), 'sampling_rate': 16000}}


In [9]:
example.keys()

dict_keys(['id', 'clean', 'noisy'])

In [10]:
clean = example['clean']
clean.keys()

dict_keys(['path', 'array', 'sampling_rate'])

In [11]:
clean['array'].shape

(36480,)

In [12]:
example['noisy']['array'].shape

(36480,)

In [14]:
type(ds['train'])

datasets.arrow_dataset.Dataset

In [22]:
ds1 = ds['train'].train_test_split(test_size=0.1, seed=42)

In [23]:
ds1.keys()

dict_keys(['train', 'test'])

In [24]:
dataset_dict = {"train": ds1['train'], "val": ds1['test'], "test": ds['test']}

In [25]:
ds1['test'].shape

(1158, 3)

In [26]:
ds['test'].shape

(824, 3)

In [4]:
temp = ds['train'].train_test_split(test_size=0.1, seed=42)
dataset = {
    "train": temp['train'],
    "val": temp['test'],
    "test": ds['test']
}

Loading cached split indices for dataset at /storage/hdd0/data/mayank_dataset/JacobLinCool___parquet/JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-4ea28409c9bd1193.arrow and /storage/hdd0/data/mayank_dataset/JacobLinCool___parquet/JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cbb577a008396ea1.arrow


In [5]:
dataset['test']

Dataset({
    features: ['id', 'clean', 'noisy'],
    num_rows: 824
})

In [6]:
for example in dataset['test']:
    print(example)
    break

{'id': 'p232_001', 'clean': {'path': 'p232_001.wav', 'array': array([ 0.00213623,  0.00323486,  0.00244141, ..., -0.00576782,
       -0.00543213, -0.00354004]), 'sampling_rate': 16000}, 'noisy': {'path': 'p232_001.wav', 'array': array([ 0.01589966,  0.02572632,  0.02059937, ..., -0.00421143,
       -0.00469971, -0.00354004]), 'sampling_rate': 16000}}


In [8]:
example['noisy'].keys()

dict_keys(['path', 'array', 'sampling_rate'])

In [10]:
example['noisy']['path']

'p232_001.wav'

In [ ]:
# read pkl file
import pickle


with open('./checkpoints/waveunet/results.pkl', 'rb') as f:
    test_data = pickle.load(f)
len(test_data)

824

In [2]:
test_data[0]

{'clean': {'SDR': array([23.12008813]),
  'ISR': array([30.36611248]),
  'SIR': array([inf]),
  'SAR': array([23.96229591])}}

In [1]:
from tensorboard.backend.event_processing import event_accumulator

# Load the event file
ea = event_accumulator.EventAccumulator("./logs/waveunet/events.out.tfevents.1744029334.gpuserver.1312197.0")
ea.Reload()

# List all tags
print("Tags:", ea.Tags())

Tags: {'images': [], 'audio': ['input', 'clean_pred', 'clean_target'], 'histograms': [], 'scalars': ['lr', 'train_loss', 'val_loss', 'test_loss', 'test_SDR_clean', 'test_SIR_clean', 'test_SDR'], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}


In [14]:
pred = ea.Audio("clean_pred")
pred

[AudioEvent(wall_time=1744037925.2935774, step=105941, encoded_audio_string=b'RIFF\xd6\xfa\x00\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80>\x00\x00\x00}\x00\x00\x02\x00\x10\x00data\xb2\xfa\x00\x00\x1f\x11\xb4\x10\xa8\x0f\x98\x0f<\x11\x0e\x11\xe0\x0f7\x10\xfd\x10\x94\x12(\x13\x19\x12\x1e\x10\t\x0fv\x10\xaa\x13\xc2\x15\xe1\x15$\x15[\x13\xd0\x0e\xb8\x08\xe8\x06\xae\x0b\x1a\x13\xbe\x12\xe5\t\x85\x03\x95\x00a\xfd\x90\xfb\xb0\xffP\x01\x95\xfdt\xf8*\xf65\xf4e\xf0\xb5\xed\xdb\xefL\xf3V\xf44\xf4y\xf2\xfc\xefK\xed~\xed\xbd\xf1J\xf6\xd9\xf7\x9d\xf9\xc7\xf9\xd1\xf7\x81\xf5\x95\xf6\x98\xfa\x9a\xfdq\xff\xb8\x01\xd8\x01\x1b\xff\x9d\xfdt\xfd\x1d\xfc\x16\xfc\xbe\xff\xe8\x00\x94\xff\t\xfc\xde\xf7Z\xf5*\xf5>\xf65\xf7\'\xf7\xce\xf5\xcf\xf2"\xf1z\xf06\xf0h\xf0\xd2\xf1\xa7\xf4\x18\xf6\x01\xf6t\xf4.\xf4\xbc\xf5j\xf9\xa8\xfcK\xff\xab\x00\xf6\xff\x13\x021\x05Q\x05\xf7\x05C\t+\x0bq\x0ch\rH\rX\x0e\x92\rl\r\x85\x0f\xf5\x10-\x11\x93\x10|\x0e\x18\x0e\xc9\x10\x1d\x12\xbf\x10\xc5\x0f\x06\x10\xdb\x0e&\x0f\xde\x10\

In [18]:
for input_, clean_pred, clean_target in zip(ea.Audio('input'), ea.Audio('clean_pred'), ea.Audio('clean_target')):
    with open(f'./audio_outputs/input_{input_.step}.wav', 'wb') as f:
        f.write(input_.encoded_audio_string)
    with open(f'./audio_outputs/pred_{clean_pred.step}.wav', 'wb') as f:
        f.write(clean_pred.encoded_audio_string)
    with open(f'./audio_outputs/target_{clean_target.step}.wav', 'wb') as f:
        f.write(clean_target.encoded_audio_string)